In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from scipy.sparse import coo_matrix, hstack


In [2]:
#ugly but functional function
def create_file(name,mis_labels,agr_labels):
    indexes=[x for x in range(1,1001)]
    data={'id':indexes,
    'misogynous':mis_labels,
    'aggressiveness':agr_labels}
    dataframe=pd.DataFrame(data,index=indexes,columns=['id','misogynous','aggressiveness'])
    dataframe.to_csv(name,sep='\t',index=False)

# Run 1 constained tf-idf

In [3]:
DATAFRAME_train = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep='\t')
DATAFRAME_test=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')
DATAFRAME=pd.concat([DATAFRAME_train,DATAFRAME_test])

vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
    strip_accents='unicode', analyzer='word', token_pattern=r'\b[^\d\W]+\b',
    ngram_range=(1, 5),use_idf=True)


In [4]:
X=vectorizer.fit_transform(DATAFRAME['text'])
X_train,X_test=X[:5000],X[5000:]
model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)

model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test)
model.fit(X_train,DATAFRAME_train['aggressiveness'])
result_agr=model.predict(X_test)



In [5]:
for i,(mis,agr) in enumerate(zip(result_mis,result_agr)):
    if mis==0 and agr==1:
        result_agr[i]=0 #Sunt 4 cazuri cand se intampla

In [6]:
create_file('MDD.A.r.c.run1',result_mis,result_agr)

# Unconstrained Run 2

In [7]:
def get_mean_embeddings(texts,embeddings):
    means = []
    dim = len(list(embeddings.values())[0])
    for text in texts :
        text = nltk.WordPunctTokenizer().tokenize(text)
        means.append(np.mean([embeddings[w] if w in embeddings else np.zeros(dim) for w in text], axis=0))
    return np.array(means)

embeddings_index = {}
with open('glove/glove.twitter.27B.200d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [22]:
DATAFRAME_train = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep='\t')
DATAFRAME_test=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')
DATAFRAME=pd.concat([DATAFRAME_train,DATAFRAME_test])

In [9]:
X=DATAFRAME['text'].values
X_train,X_test=get_mean_embeddings(X[:5000],embeddings_index),get_mean_embeddings(X[5000:],embeddings_index)

model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)

model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test)
model.fit(X_train,DATAFRAME_train['aggressiveness'])
result_agr=model.predict(X_test)

for i,(mis,agr) in enumerate(zip(result_mis,result_agr)):
    if mis==0 and agr==1:
        result_agr[i]=0 #Sunt 4 cazuri cand se intampla

create_file('MDD.A.r.u.run2',result_mis,result_agr)

## Unconstrained run 3

In [10]:
import spacy
from noun_chunks import *
import nltk

In [12]:
def clean_text(text):
    text = text.lower()
    text = text.split()

    text = [w for w in text if '@' not in w]
    text = [w for w in text if 'http' not in w]
    text = " ".join(text)

    # Tokenize each word
    text = nltk.WordPunctTokenizer().tokenize(text)
    return text

In [34]:
DATAFRAME_train = pd.read_csv('different processed dataframes/noun_chuncks,processed.tsv', sep='\t')
DATAFRAME_test=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')
# n a mers pe jupyter
# nlp = spacy.load('it_core_news_sm')

import it_core_news_sm
nlp=it_core_news_sm.load()

text_list = []
for i, text in enumerate(DATAFRAME_train['text']):
    doc = nlp(text)
    ncs = noun_chunks(doc)
    chunks = [doc[nc[0]:nc[1]].text.lower() for nc in ncs]
    text_chunks = str(" ".join(chunks))
    text_list.append(text_chunks)


DATAFRAME_train['clean'] = text_list
DATAFRAME_train['clean'] = list(map(clean_text, DATAFRAME_train['clean']))
DATAFRAME_train['clean'] = [' '.join(text) for text in DATAFRAME_train['clean']]

AttributeError: 'list' object has no attribute 'astype'

In [37]:
DATAFRAME_train['clean']=DATAFRAME_train['clean'].apply(lambda x: str(x))

In [39]:
X=DATAFRAME_train['clean'].values.astype('str')
X_train,X_test=get_mean_embeddings(DATAFRAME_train['clean'],embeddings_index),get_mean_embeddings(DATAFRAME_test['text'],embeddings_index)
model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)


model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test)
model.fit(X_train,DATAFRAME_train['aggressiveness'].values)
result_agr=model.predict(X_test)


for i,(mis,agr) in enumerate(zip(result_mis,result_agr)):
    if mis==0 and agr==1:
        result_agr[i]=0 
        
create_file('MDD.A.r.u.run3',result_mis,result_agr)

# TASK B RUN 2

In [41]:
DATAFRAME_train_synt = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_synt.tsv', sep='\t')
DATAFRAME_train_raw=pd.read_csv('AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep='\t')
DATAFRAME_train=pd.concat([DATAFRAME_train_raw,DATAFRAME_train_synt],ignore_index=True)

DATAFRAME_test_synt=pd.read_csv('test/AMI2020_test_synt.tsv',sep='\t')
DATAFRAME_test_raw=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')


In [ ]:
def get_mean_embeddings(texts,embeddings):
    means = []
    dim = len(list(embeddings.values())[0])
    for text in texts :
        text = nltk.WordPunctTokenizer().tokenize(text)
        means.append(np.mean([embeddings[w] if w in embeddings else np.zeros(dim) for w in text], axis=0))
    return np.array(means)

embeddings_index = {}
with open('glove/glove.twitter.27B.200d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [54]:

X_train,X_test_raw,X_test_synt=get_mean_embeddings(DATAFRAME_train['text'],embeddings_index),get_mean_embeddings(DATAFRAME_test_raw['text'],embeddings_index),get_mean_embeddings(DATAFRAME_test_synt['text'],embeddings_index)

model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)

#Raw test
model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test_raw)
create_file_2('MDD.B.r.u.run2',result_mis)

#Synt test
model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test_synt)
create_file_2('MDD.B.s.u.run2',result_mis)


In [53]:
def create_file_2(name,mis_labels):
    indexes=[x for x in range(1,len(mis_labels)+1)]
    data={'id':indexes,
    'misogynous':mis_labels}
    dataframe=pd.DataFrame(data,index=indexes,columns=['id','misogynous'])
    dataframe.to_csv(name,sep='\t',index=False)